# Summary

Today, we're going to be implementing least squares regression subject
to an *increasingness* constraint. That is, we're going to find a curve
$\hat \mu$ that solves this optimization problem.

$$
\hat \mu = \operatorname*{argmin}_{\text{increasing} \ m} \frac{1}{n}\sum_{i=1}^n \{ Y_i - m(X_i) \}^2.
$$

Then we'll see what it tells us about the effect of reducing class
sizes for 5th graders using some admittedly fake data about test outcomes.


We'll use a few libraries.

In [ ]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(CVXR)
})

# OSQP claims some feasible problems aren't, so we'll tell CVXR not to use it
CVXR::add_to_solver_blacklist('OSQP')  

# And we'll style our plots  
theme_update(plot.background = element_rect(fill = "transparent", colour = NA),
		    panel.background = element_rect(fill = "transparent", colour = NA),
                    legend.background = element_rect(fill="transparent", colour = NA),
                    legend.box.background = element_rect(fill="transparent", colour = NA),
                    legend.key = element_rect(fill="transparent", colour = NA),
			panel.grid.major=element_line(color=rgb(1,0,0,.1,  maxColorValue=1)),
	        panel.grid.minor=element_line(color=rgb(0,0,1,.1,  maxColorValue=1)),
		    axis.ticks.x = element_blank(),
		    axis.ticks.y = element_blank(),
		    axis.text.x  = element_text(colour = "#aaaaaa"),
		    axis.text.y  = element_text(colour = "#aaaaaa"),
		    axis.title.x  = element_text(colour = "#aaaaaa"),
		    axis.title.y  = element_text(colour = "#aaaaaa", angle=90))

We'll start by focusing on the increasing case. We'll use `CVXR` to help us solve the following optimization problem.

$$
\hat \mu = \operatorname*{argmin}_{\text{increasing} \ m} \frac{1}{n}\sum_{i=1}^n \left\{ Y_i - m(X_i) \right\}^2.
$$

This might feel like a tall order because there are *a lot* of increasing functions. The set is infinite-dimensional, which essentially means you can't play 20 questions. 

- If you know I'm thinking of an increasing function, there's no number of questions you can ask me that'll let you pin down which one. 
- If I wanted to cheat at the game, every time you said 'is it this one' I could come up with an increassing function that's different from the one you guessed *and* consistent with all the answers I've given you so far.

But it turns out that finding a solution to optimization problem is pretty easy if you break it down into steps in the right way. We're going to use an approach I'll call **restriction and extension**, which often works when you're trying to solve infinite dimensional optimization problems.

# Preliminary Pedantry 

## Functions, Restriction, and Extension

When we say *an increasing function*, we tend to think of an unbroken curve that goes up as you move from left to right. Like the curve $f(x)=e^x$ that we've coded up and plotted below. 

In [ ]:
x = seq(-1,1,by=.01)
f = function(x) { exp(x) }
ggplot() + geom_line(aes(x=x, y=f(x)), color='blue')

- To be precise, $f$ is a *function on the real line* because it tells us how to take any *input* $x$ on the real line to an *output* $f(x)$. 
- To be even more precise, $f$ is a function *from* the real line *to* the real line, because those outputs are real numbers too.

Here's another increasing function $g$. It's a function on a set of five points $\mathcal{X} = \{-1, -\frac12, 0, +\frac12, +1\}$. Here's how we might code and plot this.

In [ ]:
X  = c(-1,-1/2,0,1/2,1) 
e = exp(1)
g = function(x) { 
    case_when(x == -1    ~  1/e,
              x == -1/2  ~  sqrt(1/e),
              x == 0     ~  1,
              x == +1/2  ~  sqrt(e),
              x == +1    ~  e)
}

ggplot() + geom_point(aes(x=X, y=g(X)), color='red')


Because each valid input to $g$ is a valid input to $f$ as well, i.e. because the set of five points $\mathcal{X}$ is a *contained in* the set of all real numbers, we can ask if the outputs $f(x)$ and $g(x)$ coincide on $\mathcal{X}$. And they do.

In [ ]:
ggplot() + geom_line(aes(x=x, y=f(x)), color='blue') + 
           geom_point(aes(x=X, y=g(X)), color='red') 
           

But they aren't the same thing. The function $f$ is defined for all real numbers, while the function $g$ is only defined for the five points in $\mathcal{X}$. 

In [ ]:
f(3/4) 

In [ ]:
g(3/4)

When the this happens---when $f(x)$ and $g(x)$ coincide on a set $\mathcal{X}$ but $g$ is not defined elsewhere---we say that ...
- $g$ is **the** *restriction* of $f$ to $\mathcal{X}$
- $f$ is **an** *extension* of $g$ to the real line.
Those are the formal terms. 

There are other extensions of $g$. For example ...
- a **piecewise linear** extension. It's what we get by connecting the dots in the plot of $g$ above with straight lines segments. That's what ggplot's geom_line does.
- a **piecewise constant** extension. We get one by moving horizontally rightward from each dot until we hit the next one. That's what ggplot's geom_step does.

In [ ]:
ggplot() + geom_point(aes(x=X, y=g(X)), color='red') + 
           geom_line(aes(x=X, y=g(X)),  color='red', linetype='dashed') +
           geom_step(aes(x=X, y=g(X)),  color='red', linetype='dotted')


We're not quite getting extensions of $g$ to the real line when we use geom_line and geom_step. We're getting extensions to the unit interval, the range between the largest and smallest point in $\mathcal{X}$. 

**R** has a built-in function `approxfun` that will extend functions from a set of points to the real line. 

  - We can ask for piecewise-constant and piecewise-linear extensions. 
  - But the piecewise-linear extension isn't the one we usually want. 

We'll write our own extension code later on in this lab so we can get it to do what we want. What's wrong with `approxfun`? 

In [ ]:
g.linear   = approxfun(X,g(X), rule=2, method='linear')
g.constant = approxfun(X,g(X), rule=2, method='constant')

x = seq(-2,2,by=.01)
ggplot() + geom_point(aes(x=X, y=g(X)),  color='red') + 
  geom_line(aes(x=x,  y=g.linear(x)),    color='red', linetype='dashed') +
  geom_line(aes(x=x,  y=g.constant(x)),  color='red', linetype='dotted') +
  geom_line(aes(x=x,  y=f(x)),           color='blue')

### Terminology and Notation

- We say $f$ is *a function from $\mathcal{X}$ to $\mathcal{Y}$* if, for every point $x$ in 'set of possible inputs' $\mathcal{X}$, we know the corresponding value $f(x)$ and it is in the 'set of possible outputs' $\mathcal{Y}$. 
  - To make this a little more compact, often people write $f: \mathcal{X} \to \mathcal{Y}$ with this meaning. 
  - Typically this is prounounced exactly the same way, i.e. as  '$f$ is a function from $\mathcal{X}$ to $\mathcal{Y}$'.
  - Or more efficiently as '$f$ *maps* $\mathcal{X}$ to $\mathcal{Y}$'.
- If we expect the set of possible outputs to be inferred from context, we might say '$f$ is a function on $\mathcal{X}$', which is a little shorter. 
    - That's a pretty safe bet in this class because our outputs are almost always real numbers.
    - As far as I know, there isn't really an accepted notation for this. People still write $f: \mathcal{X} \to \mathcal{Y}$ even when the output set is clear from context.
    - The options $f: \mathcal{X} \to$ or $f: \mathcal{X} \to [\text{you figure it out}]$ are awkward and aggressive respectively, and while $f: \mathcal{X}$ is a little better, I don't see it much.
- When we want to write about a relationship like the one between $f$ and $g$, we often write $f|_{\mathcal{X}}$, which is pronounced 'the restriction of $f$ to $\mathcal{X}$'. The statement $g=f|_{\mathcal{X}}$ is read as '$g$ is the restriction of $f$ to $\mathcal{X}$'.
- I often find myself saying $f$ and $g$ *agree on* $\mathcal{X}$, by which I mean that $f(x)=g(x)$ for all $x$ in $\mathcal{X}$. 
    - This means restrictions of $f$ and $g$ to $\mathcal{X}$ are the same, i.e. that $f|_{\mathcal{X}} = g|_{\mathcal{X}}$, but doesn't tell us that one is a restriction of the other.
    - If we think about our code for $f$ and $g$ above, the distinction is that that $g$ might not 'return NA' for inputs $x$ that aren't in $\mathcal{X}$.  It might just return a value that's not $f(x)$.
    - For example, $f$, $g$, and the piecewise-constant and piecewise-linear extensions of $g$ all agree on $\mathcal{X}$. Their restrictions to $\mathcal{X}$ are all the same function: $g$.
- Sometimes people use different words for extension inside and outside the range of $\mathcal{X}$.
  - *Interpolation* is the process of extending a function to points inside the range of $\mathcal{X}$. That's what `geom_line` and `geom_step` did for us above.
  - *Extrapolation* is the process of extending a function to points outside the range of $\mathcal{X}$. That's what we had to use `approxfun` for. 

### Exercise
Write a function $h$ that agrees with $f(x)=e^x$ on the unit interval $[-1,+1]$ but is constant elsewhere. Plot $h$ and $f$ on the same graph.

## Increasingness 

If we have a function $f$ on a set of points $\mathcal{X}$ on the real line, we can ask if it's increasing. 

- Intuitively, that means that, as we move from left to right, the values of $f$ get bigger. Or actually, because this turns out to be more convenient, that they don't get smaller. 
- Formally, we say the $f: \mathcal{X} \to \mathbb{R}$ is increasing if $f(x) \le f(x')$ for all pairs of points $x,x'$ in $\mathcal{X}$ with $x \le x'$.

All of the functions we've seen so far are increasing. 

- The ones with flat segments, like $h$ and the piecewise-constant extension of $g$, are sometimes called *non-decreasing* instead of increasing.
    - This aligns a bit better with plain english, but it tends to make your sentences awkward and harder to understand.
    - Math folks tend to say 'strictly increasing' if they want to rule out flat segments, but it's pretty rare that they want to. 
    - Informally, I'll say 'gets bigger' meaning 'doesn't get smaller' analogously.
    
An *increasing function on the real line* ($\mathbb{R}$) is a function on the real line that gets bigger whenever $x \in \mathbb{R}$ does. That is, its values satisfy 

$$ 
f(x) \le f(x') \text{ for all pairs of points \textbf{on the real line} satisfying } x \le x' 
$$

An *increasing function on the set* $\mathcal{X}$ is one where this value gets bigger whenever $x \in \mathcal{X}$ does. That is, its values satisfy 

$$ 
f(x) \le (x') \text{ for all pairs of points \textbf{in $\mathcal{X}$} satisfying $x \le x'$} 
$$

There are two things you should know about increasingness and restriction/extension. Suppose $\mathcal{X}$ is a set of real numbers, i.e., a subset of the real line.

1. If $f$ is an increasing function on the real line: $f_{\mathcal{X}}$, *the restriction* of $f$ to $\mathcal{X}$, is increasing.
2. If $g$ is an increasing function on $\mathcal{X}$: *it has* an increasing extension to the real line. That is, there is an increasing function $f$ on the real line that agrees with $g$ on $\mathcal{X}$. 

1. If $f(x) \le f(x')$ for all pairs of real numbers $x,x'$ with $x \le x'$, then $f(x) \le f(x')$ for all pairs of real numbers with $x \le x'$ **hat are both in the set $\mathcal{X}$*.
2. The piecewise-constant extension of $g$ is increasing. 

### Exercise

Is it true that *every* extension of an increasing function $g$ on $\mathcal{X}$ is increasing? 

  - If so, explain why. 
  - If not, draw a counterexample.

### Enough Already (Almost)

If you think all this sounds a like a waste of time, I'm sympathetic. I don't usually talk this formally and I certainly don't usually write 7 lines of code when `g=exp` will do. But sometimes being formal about stuff like this can help focus our thinking.

**Question**. If I'm thinking of an increasing function on $\mathcal{X}=\{-1,-\frac12,0,+\frac12,+1\}$, or any function on $\mathcal{X}$, how many questions would you need to ask to pin down which one I'm thinking of? 

**Answer**. 5. You can just ask me for the value of the function at each point in $\mathcal{X}$.

What this tells us is that, even if it's impossible to find out exactly what increasing function $f$ on the real line I'm thinking of, it can be pretty easy to find out its restriction $f_{\mathcal{X}}$ to a finite set of points. With this in mind, let's take a look at our optimization problem again.


# Solving the Least Squares Problem 
## Step 1. Restrict and Solve

Let's take another look at the least squares problem we've been writing. It's a little vague.

$$
\hat \mu = \operatorname*{argmin}_{\text{increasing} \ m} \frac{1}{n}\sum_{i=1}^n \left\{ Y_i - m(X_i) \right\}^2.
$$

What's vague about it? It doesn't specify *what kind of function* the solution $\hat\mu$ is supposed to be, i.e., the sets of possible inputs and outputs. 

That vagueness speaks to a tension between what we want and what we can know.

- The data we have is only informative about the values of the function on the sample---the set $\mathcal{X}=\{X_1,\ldots,X_n\}$.
    - If two functions on the real line agree on the sample, then either they're both solutions to this optimization problem or neither is. 
    - This means that, at best, we can hope to identify a function on the sample.
    - In a way, this is a good thing. It means we don't have to solve an infinite-dimensional optimization problem. Computers like that.
- Often, we will want to make predictions for points that aren't in the sample, so we'll want to find a function on the real line. 
    - This means that, whatever solution we do get, we'll have to extend it to the real line. 
    - And the data doesn't prefer any one extension to another. That's on us to make up.
    - This isn't great, but it's not damning either. In large samples, the extension we use doesn't matter much. Why?
    
Here's the recipe for solving this optimization problem.
1. Think of it as a problem of choosing from the set of increasing functions on the sample.
    - That's a finite-dimensional model: we need one parameter for each distinct value of $X_1 \ldots X_n$.
    - We can solve it using CVXR, just like we did for the linear regression problem.
2. Once we have a solution, we can extend it to an increasing function on the real line in any way we like. The data doesn't care.
    - We can, for example, do piecewise constant extension or piecewise linear extension.
    - If we have an increasing function on the sample, these extensions will be increasing functions on the real line.
    
Let's solve for a function on the sample first.  We'll worry about the extension later.  


### Exercise: Implementation in Mathematical Notation

To keep things simple we'll start by assuming that the observed values of $x$, $X_1 \ldots  X_n$, are distinct. 

`CVXR` likes to work with vectors, not functions on finite sets, so we'll have to do a little translation.

 - We'll tell `CVXR` we're interested in optimizing over vectors $\vec m$ in some set $\vec M \subseteq \mathbb{R}^n$. 
 - We'll interpret these vectors as functions $m:\mathcal{X} \to \mathbb{R}$  using the correspondence $m(X_i) = \vec m_i$. 

Plugging this correspondence into our optimization problem, we get ...

$$
\begin{aligned}
\hat \mu(X_i) &= \vec\mu_i  
&&  \text{ where } \\
&\vec \mu = \operatorname*{argmin}_{\vec m \in \vec M}  \frac{1}{n}\sum_{i=1}^n \{ Y_i - \vec m_i \}^2  
&& \text{ for } \\ 
& \textcolor{red}{\vec M = \text{some set of vectors in} \  \mathbb{R}^n}.
\end{aligned}
$$

If we want $\hat\mu$ to solve the optimization problem we started with, i.e. $\hat\mu = \operatorname*{argmin}\limits_{\text{increasing} \ m: \mathcal{X} \to \mathcal{R}} \frac{1}{n}\sum_{i=1}^n \left\{ Y_i - m(X_i) \right\}^2$, what should $\textcolor{red}{\vec M}$ be?

**Tips**.

1. Look back at the definition of increasingness and think about what it means for a function on $\mathcal{X}=\{X_1 \ldots X_n\}$. Finish this sentence. 
 > $m:\mathcal{X} \to \mathbb{R}$ is increasing *if and only if* the $n$ values $m(X_1) \ldots m(X_n)$ satisfy ...
2. If you've expressed what you want in terms of the $n$ values $m(X_1) \ldots m(X_n)$, you can translate it into a statement about $\vec m$ mechanically. Just replace $m(X_i)$ with $\vec m_i$.


### Exercise: Implementation in `CVXR`

Write some monotone regression code. Then use it to plot $\hat\mu(X_i)$ on top of the data below. 

I've written a template for you to fill in. 
  - If you run it without changes, it'll give you *an* increasing function, by which I mean a vector that satisfies the constraints 
in the optimization problem written out above. 
  - But not necessarily one that's a good fit to the data. 
  - Go ahead and make changes where indicated to get the right fit.

There are a few tips below the template.

In [ ]:
monotonereg = function(X,Y) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(!anyDuplicated(X))
  stopifnot(length(X)==length(Y))
  n = length(X)

  # Step 1. 
  # We tell CVXR we're thinking about a vector of unknowns m in R^n.
  m = Variable(n)

  # Step 2. 
  # We tell CVXR that we're interested in mean squared error.
  mse = sum(m^2)                     ## CHANGE THIS

  # Step 3. 
  # We specify our constraints.
  constraints = list(m == 0)         ## CHANGE THIS

  # Step 4. 
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R. 
  #  1. we record the vector of inputs, X, and the vector of corresponding outputs, mu.hat, in a list
  #  2. we assign that list a class, so R knows predict should delegate to predict.monotonereg
  #  3. we return the list
  model = list(X=X, mu.hat=mu.hat)
  attr(model, "class") = "monotonereg"
  model
}

In [ ]:
monotonereg = function(X,Y) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(!anyDuplicated(X))
  stopifnot(length(X)==length(Y))
  n = length(X)

  # Step 1. 
  # We tell CVXR we're thinking about a vector of unknowns m in R^n.
  m = Variable(n)

  # Step 2. 
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y-m)^2)/n                                       ## CHANGED 

  # Step 3. 
  # We specify our constraints.
  all.pairs = expand.grid(i=1:n, j=1:n)
  le = all.pairs$i <= all.pairs$j 
  le.pairs = all.pairs[le,]
  constraints = list(m[le.pairs$i] <= m[le.pairs$j])         ## CHANGED

  # Step 4. 
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R. 
  #  1. we record the vector of inputs, X, and the vector of corresponding outputs, mu.hat, in a list
  #  2. we assign that list a class, so R knows predict should delegate to predict.monotonereg
  #  3. we return the list
  model = list(X=X, mu.hat=mu.hat)
  attr(model, "class") = "monotonereg"
  model
}


**Tips.**

1.  `CVXR` takes a list of constraints expressed in terms of a vector of unknowns `m` that it calls a Variable.
    - These are expressions that would evaluate to TRUE or FALSE if `m` were a vector of numbers, e.g. `m[i] <= m[j]` for some pair of indices `i` and `j`.  
    - It's also happy if these constraints in this list are *vector-valued expressions*, i.e. expressions that would evaluate to a vector with elements that are TRUE or FALSE. 
      - In this case, it enforces the constraint that all elements of the vector are TRUE when it solves for `m`. 
      - This means that if you've got *vectors* of indices `i` and `j`, saying `m[i] <= m[j]` will enforce that `m[i[1]] <= m[j[1]]`, `m[i[2]] <= m[j[2]]`, etc.

2.  If you want to make a list of all pairs of indices $i$ and $j$ with $X_i \le X_j$, you can do it like this.
    - Make a table of all pairs of indices. 
    - Find the rows where $X_i \le X_j$. 

    Some code that does this is below.

In [ ]:
X = c(-1,-1/2,0,1/2,1)
n = length(X)

all.pairs=expand.grid(i=1:n,j=1:n)
le = X[all.pairs$i] < X[all.pairs$j]
le.pairs = all.pairs[le,]

i = le.pairs$i
j = le.pairs$j

i
j

### Visualizing Your Solution

Here's some code to help you see what yours is doing. It uses the code you've written to fit a function to some data
and then plots the function and data together. We'll use data sampled around our old friend from the warm-up, the 'stepline'. 
It'll plot observations $(X_i, Y_i)$ in as gray dots and the predictions $(X_i, \hat\mu(X_i))$ as blue ones. And because 
this is fake data, we can plot the function we've sampled our points around, too. That'll be a black line.

You don't need to change this code at all, just run it after you've written what you need to above. If you run this code
without changing *that*, $\hat\mu$ will be some increasing function but not necessarily a good one. That said, it might
be good in some places and not others---a stopped clock is right twice a day.

If you've written the code above correctly, you should get a pretty good estimate of the black line. 
It is, after all, an increasing function. 

In [ ]:
# data
mu = function(x) { x*(x >= .5) }
sigma = .1

n = 100
X = seq(0,1,length.out=n)
Y = mu(X) + sigma*rnorm(n)

# our fit curve
model   = monotonereg(X,Y)

# a grid for plotting and a plot
x = seq(0,1,by=.001) 
fit.on.sample = ggplot() + 
	geom_point(aes(x=X,y=Y),  alpha=.2) + 
	geom_line(aes(x=x,y=mu(x)), alpha=1) +
	geom_point(aes(x=model$X,y=model$mu.hat), color='blue', alpha=.2)
fit.on.sample

### Exercise: Exploration

1. Try changing the function `mu` that we're sampling our data around. Find a 'hard function' one that your estimate $\hat\mu$ doesn't fit well.
2. Try changing the sampling size `n`. If you make it bigger, does your fit to the 'stepline' improve significantly? What about your 'hard function'?
3. Try fitting a version of the data without noise, i.e., `Y.without=mu(X)`. 
    - Vary `n` and compare to the estimate you get when you fit the noisy data. 
    - Do this for the 'stepline' and your 'hard function'. What do you notice?

### Exercise: Optimization
 
Now let's simplify our set of constraints to make things easier for
`CVXR`. When we have a constraint for all pairs of indices $i,j$ with
$X_i \le X_j$, many of them are redundant. 

- This is a consequence of *transitivity*: the constraints $m(X_i) \le m(X_j)$ and
$m(X_j) \le m(X_k)$ imply the additional constraint $m(X_i) \le m(X_k)$.
- It means that if our points $X_i$ are sorted in increasing order, i.e. if $X_1 \le X_2 \le \ldots \le X_{n-1} \le X_n$, the
constraints $m(X_{i}) \le m(X_{i+1})$ for $i \in 1 \ldots n-1$ imply the
whole set.

Here's a template to modify.  I've done the sorting for you so
you can focus on formulating your optimization in terms of the sorted
data. What you need to do is fix the mse and monotonicity constraint in lines 19 and 23.

In [ ]:
monotonereg.fast = function(X,Y) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(!anyDuplicated(X))
  stopifnot(length(X)==length(Y))
  n = length(X)
  # and reorder pairs (Xi,Yi) so Xs are sorted: X[i] <= X[i+1] <= ...
  X.orig = X
  increasing.order=order(X)
  Y = Y[increasing.order]
  X = X[increasing.order]
  
  # Step 1. 
  # We tell CVXR we're thinking about a vector of unknowns m in R^n.
  m = Variable(n)

  # Step 2. 
  # We tell CVXR that we're interested in mean squared error.
  mse = sum(m^2)                     ## CHANGE THIS. X AND Y ARE IN CORRESPONDENCE AND X IS SORTED IN INCREASING ORDER

  # Step 3. 
  # We specify our constraints.
  constraints = list(m == 0)         ## CHANGE THIS. X AND Y ARE IN CORRESPONDENCE AND X IS SORTED IN INCREASING ORDER

  # Step 4. 
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  
  # Step 5: a little boilerplate to make it idiomatic R. 
  #  1. we record X and mu.hat, in correspondence and sorted in increasing order of X, and X in its original order, in a list
  #  2. we assign that list a class, so R knows predict should delegate to predict.monotonereg
  #  3. we return the list
  model = list(X=X, mu.hat=mu.hat, X.original.order=X.orig) 
  attr(model, "class") = "monotonereg"
  model
}

You can test that your new code does the same thing as the old stuff by running the block below. If you've done everything right, 
you should get essentially the same answer from both versions of the code. There might be some small differences due to rounding error.

In [ ]:
model = monotonereg(X,Y)
model.fast = monotonereg.fast(X,Y)
max.difference = max(abs(as.vector(model$mu.hat)-as.vector(model.fast$mu.hat)))
max.difference

### Exercise: Generalization

The code you're written above has a few limitations relative to what you might want in practice.

1. There's no option to fit a decreasing function instead of an increasing one.
2. It requires the observations $X_1 \ldots X_n$ to be distinct.

Let's address these so we get code that's a little more broadly applicable. 

The decreasing bit is easy. You'll need to tweak to the constraints you're passing to `CVXR`, but it's probably not too hard to figure out what the change should be.

Dealing with non-distinct $X_i$ is a little trickier, so let's take a minute to prepare. The issue is that, if $X_1 \ldots X_n$ contains duplicates,
we can't think of $m(X_1) \ldots m(X_n)$ as $n$ unknowns: if $X_i=X_j$, then $m(X_i)$ and $m(X_j)$ have to be the same. Handling this is just a matter
of bookkeeping. If $X_1 \ldots X_n$ has $p$ distinct values, then that's how many unknowns we need to solve for. And what we need is to know, for each $X_i$,
which of these unknowns it corresponds to. To help out, I'll give you a function `invert.unique` that takes a vector $X$ and returns a list of two things.

1. A vector `elements` cointaining the positions of the unique elements of $X$, so that `X[elements]` is vector of unique elements of $X$ sorted in increasing order. 
    - This vector has length $p$ where $p$ is the number of unique elements in $X$.
2. A vector `inverse` that tells you, for each $i$, the position of $X[i]$ in $X[elements]$. 
    - This vector has length $n$ where $n$ is the length of $X$.
    - `X[elements][inverse]` is the same as `X`.

A lot of programming languages have a function like this built in---*Matlab* does and *Python* (NumPy) does---but `R` doesn't. It's a bit fussy to write, so I've saved you the trouble.

In [ ]:
invert.unique = function(x) { 
  o = order(x)
  dup = duplicated(x[o])
  inverse = rep(NA, length(x))
  inverse[o] = cumsum(!dup)
  list(elements=o[!dup], inverse=inverse)
}

a = c(1,2,3,3,4,5,5,5,6)
unique.a=invert.unique(a)
stopifnot(a[unique.a$elements][unique.a$inverse] == a)

Go ahead and write a function `monotonereg` that works with non-distinct $X_i$. This isn't a big change to the code you just wrote above, but you may need to think a little bit about how to use `invert.unique` to make it work.
When you think you've got it, test it out and make sure it works as you expect. You can use the block above to check that it does the right thing *without* duplicates, but you're on your own for checking that it works *with* duplicates.

When you've got that working, test it out and make sure it works as you expect. You can use the block above to check that it does the right thing *without* duplicates, but you're on your own for checking that it works *with* duplicates.
Then, tweak it so it fits a decreasing function instead of an increasing one if you pass `decreasing=TRUE`. Go ahead and test that out, too.

I'm not going to give you a template for this one because I can't think of a way to do it that doesn't totally give it away, but I'll talk you through it . You'll want to tell `CVXR` that you're thinking about a vector $m$ of $p$ unknowns, where $p$ is the number of unique elements in $X$. Go ahead an think of $m[1]$ as the function's value at the smallest level of $X_i$, $m[2]$ as the function's value at the second smallest level of $X_i$, etc. It should be pretty easy to work out what the right constraints are in terms of the vector $m$. From there, all you need to know is which position in $m$ corresponds to each *observation* $(X_i,Y_i)$, so you can use the right value of $m$ in the loss function. The `inverse` you get from `invert.unique(X)` should help you with that.

All that said, while it's useful to be able to do little programming tasks like this, that's not really what this class is supposed to be about. If you're having trouble with this or you don't feel motivated, go ahead and skip it. 
It's not going to be something you'll need later on in class and I'll give you code that does all this in the solution.

# Step 2. Extending Your Solution




Clearly there are functions that are increasing on the sample that
aren't, in fact, increasing. Draw one on this sample.

```{r}
set.seed(1)
x=seq(.15,.85,by=.1)
y=x+rnorm(x)
ggplot() + geom_point(aes(x=x, y=y)) + xlab('') + ylab('')
```

However, given any function $m$ that is increasing on the sample, there
is an everywhere increasing function that agrees with it *on the
sample*. In fact, there are infinitely many. Draw one.

The first curve you drew probably wasn't a solution to the optimization
problem above. That's not something I can just draw, anyway. I need the
code we're writing in this lab to do it. But if you had drawn a solution
to the optimization problem above, then drawn an increasing curve that
agrees with it on the sample, you'd be set. Since the optimization
problem above depends only on the values of $m$ on the sample, that
increasing curve would be another solution to that problem. And because
it's an increasing solution to our increasing-on-the-sample least
squares problem, it's a solution to the outright-increasing least
squares problem we set out to solve.

That means we can solve the increasing least squares problem in two
steps. First, we'll find a solution $\hat\mu_{sample}$ to the
increasing-on-the-sample problem. Then, we'll draw an increasing curve
$\hat \mu$ that agrees with $\hat\mu_{sample}$ on the sample.

Finding an Increasing Solution

## Sketching

Now we're going we're going to use our solution $\hat\mu_{sample}$ from
the previous part to find a curve $\hat \mu$ that solves the increasing
least squares problem. By this, we mean a function $\hat\mu$ that we can
evaluate at any $x \in \mathbb{R}$. Then we'll plot the curve
$\hat\mu(x)$ for $x \in [0,1]$. This will connect the points you plotted
in the previous part.

In Section \@ref(breaking-the-problem-down), we argued that any
increasing curve $\hat \mu$ that goes through the points
$\{(X_i,\hat\mu_{sample}(X_i) \}$ solves the increasing least squares
problem. So all we need is code that takes an increasing point-set, i.e.
a set $\{(X_i, Y_i) : i\le n\}$ for which $Y_i \le Y_j$ whenever
$X_i \le X_j$, and evaluates $m(x)$ for an increasing curve $m$ that
goes through these points.

We did this by hand in Section \@ref(breaking-the-problem-down). Now we
need to come up with an algorithm that does it. Writing code than runs
on computers is hard, so to ease into this, let's write code that runs
on your classmates.

Write down an algorithm for drawing an increasing curve that goes
through an increasing point-set $\{(X_i, Y_i) : i \le n\}$. You don't
have to write pseudocode. Think of it as giving instructions two
different people could follow to draw the curve on top of a plot of the
points $\{(X_i, Y_i)\}$. Make sure that your curve extends past the
edges of your data. We want our increasing curve defined for all $x$ on
the real line.


As a test, use your algorithm to draw a curve through the following two
plots of an increasing point-set $\{(X_i, Y_i)\}$. Then give your
algorithm to a classmate and ask them to use it to do the same. If there
are any differences between what you draw and they do, one of two things
went wrong. Either your algorithm was too vague to have a well-defined
outcome or somebody made an error following it. Figure out which and
eliminate any vagueness. Soon you'll be implementing this on a computer,
which can't handle vague code.


In [ ]:
x=seq(.05,.95,by=.1)

y=1-cos(pi*x/2)
ggplot(data.frame(x=x,y=y), aes(x=x, y=y)) + geom_point() + ylim(0,1)

y=((1+x)/2)*(.3 + .5*(x >= .5) + .2*(x >= .8))
ggplot(data.frame(x=x,y=y), aes(x=x, y=y)) + geom_point() + ylim(0,1)


## Implementation

Once you've got that working, implement a `predict` function like
`predict.linearreg` to fill in the gaps. Then add the curve to your plot
from the last exercise.

**Tips.**

1.  It's likely that your algorithm for drawing an increasing curve
    through a point-set $\{(X_i, Y_i) : i \le n\}$ involves knowing the
    closest point $X_i$ to the left of each $x \in \mathbb{R}$, or to
    the right of it, or both. To make this easier, I recommend you sort
    everything so $X_1 \le X_2 \le \ldots \le X_n$ throughout.

2.  When making your predictions, you can use the built-in `R` function
    `findInterval` to find the index $i$ of the closest point to the
    left of $x$. In particular, if $X$ is the sorted vector
    $[X_1 X_2 \ldots X_n]$ and $x$ is a vector of query points,
    `findInterval(X,x)` will return a vector $i$ such that $X_{i_k}$ is
    the closest point to the left of $x_k$. Naturally $X_{i_k+1}$ will
    be the closest point to the right of $x_k$. You may have to handle
    some edge cases. If there is no observation $X_i$ left of $x_k$,
    then the entry $i_k$ will be zero; if there is no observation
    $X_{i}$ to the right of it, $i_{k}+1$ will be $n+1$.

Here's a template. To help you out, I'll give you code to put your
observations $X_i$ and corresponding predictions $\hat\mu(X_i)$ in
sorted order.

In [ ]:
predict.monotonereg = function(model, newdata=data.frame(X=model$X)) {
  # Get X and mu.hat in sorted order
  increasing.order = order(model$X) 
  X = model$X[increasing.order]
  mu.hat = model$mu.hat[increasing.order]
  
  # Do some stuff to compute predictions for each observation in newdata$X
  
  # Return these predictions as a vector
  
}

Having implemented this predict method, let's use it to plot a curve.
We'll add this curve to the plot of our observations and our predictions
$\hat\mu(X_i)$ that we made in the last section.

In [ ]:
x = seq(0,1,by=.001)
muhat.x = predict(model, newdata=data.frame(X=x))
fit.on.sample + geom_line(aes(x=x, y=muhat.x)) 


# Application

Now let's see what this tells us about the effect of class sizes on 5th
graders' test scores in Israel. Almost all of the code in this section
is all written for you. Your job is interpreting the results.

We'll start by grabbing some data from my website and plotting it.

```{r load-israel-schools-data}
school=read.csv('https://davidahirshberg.bitbucket.io/data/israel-schools-enrollment-model-1200-points.csv')
ggplot() + geom_point(aes(x=enrollment, y=score), alpha=.2, data=school)
```

Recall from our first lecture that class sizes are capped at 40, so
there are $x$ students per class in schools with enrollment $x \le 40$,
an average of $x/2$ in schools with enrollment $x \in [41,80]$, an
average of $x/3$ in schools with enrollment $x \in [81,120]$, and so on.
To keep things simple, we'll just be comparing schools with enrollment
$x \le 80$. We'd expect test scores to *decrease* as a function of class
size and therefore, if we stick to one of the ranges $[1,40]$ and
$[41,80]$, with enrollment. That means we'll need to alter our code
above so we can fit decreasing curves as well.



Go back to the function `monotonereg` that you've written above and
generalize it so that it fits a decreasing curve rather than an
increasing one if passed the argument `decreasing=TRUE`.


Here's our estimate, $\hat \mu_{right}(40) - \hat \mu_{left}(40)$, of
the effect if decreasing classes from size $40$ to size $40/2=20$. Since
I'm not giving you monotone regression code, I'll use linear regression.
You'll change that.

In [ ]:
linearreg = function(X,Y) { lm(Y~X) }

estimate.effect = function(fit.model) { 
  data.left  = school[school$enrollment <= 40, ]
  data.right = school[40 < school$enrollment & school$enrollment <= 80, ]

  model.left  = fit.model(data.left$enrollment,   data.left$score)
  model.right = fit.model(data.right$enrollment,  data.right$score)

  effect.estimate = predict(model.right, newdata = data.frame(X=40)) - 
                    predict(model.left, newdata  = data.frame(X=40))
  
  list(model.left  = model.left, 
       model.right = model.right, 
       data.left   = data.left,
       data.right  = data.right,
       estimate=effect.estimate)
}

estimate = estimate.effect(linearreg)
estimate$estimate


And let's plot our fits $\hat\mu_{left}$ and $\hat\mu_{right}$ over the
data to get a sense of what our estimate is based on.


In [ ]:
plot.effect.estimate = function(estimate) { 
  x.left  = seq(0,40, by=.1)
  x.right = seq(40,80,by=.1)
  plot.left  = data.frame(enrollment=x.left,  
                          score=predict(estimate$model.left,  newdata=data.frame(X=x.left)))
  plot.right = data.frame(enrollment=x.right, 
                          score=predict(estimate$model.right, newdata=data.frame(X=x.right)))
  ggplot() + geom_point(aes(x=enrollment, y=score), alpha=.2, 
                        data=rbind(estimate$data.left, estimate$data.right)) +
	           geom_line(aes(x=enrollment,  y=score), color='blue',  data=plot.left) +
	           geom_line(aes(x=enrollment,  y=score), color='blue',  data=plot.right) +
	           geom_vline(aes(xintercept=40)) 
}
plot.effect.estimate(estimate)

When you've got your monotone regression code code written, use it to estimate to treatment effect. Then illustrate what it's based on by plotting it as above.

Compare the effect estimates we get using linear regression and
decreasing regression. Which estimate are you inclined to trust? Why?


Now let's check that we're getting the same answer we did before.
Because we're using computer arithmetic, which is a bit imprecise, you
shouldn't expect exact equality of the vectors
$\hat\mu(X_1) \ldots \hat\mu(X_n)$ you get with these different
implementations, but they should be close.


# Conclusion

That speaks to a more general approach we can use for any model.

1. Work out the *set of functions on the sample*, $\mathcal{X}|_{\mathcal{X}}$, that agree with functions in the model.
   $$ \mathcal{M}|_{\mathcal{X}} = \{ m|_{\mathcal{X}} : m \in \mathcal{M} \} $$
   Because there are only finitely many observations, this is a finite-dimensional set. Often, we can optimize over it using CVXR.
2. Extend your solution to a function that's actually in the model however you like.
    - This can be pretty mechanical, but it is possible to screw it up. Let's look at how.
agree with functions in the model.





# A Variation

If you have extra time, check out [nearly-monotone
regression](https://www.stat.cmu.edu/~ryantibs/papers/neariso.pdf).[^1]
The nearly-monotone model adds a bit of slack so you can fit curves that
aren't monotone, but are close. This is a version for nearly-increasing
curves. The parameter $B$, which is up to you, controls how much slack
is allowed. 

$$
\hat \mu = \operatorname*{argmin}_{\substack{m \\ \sum_{i=1}^{n-1} \{m(X_{i})-m(X_{i+1})\}_+ \le B}} \frac{1}{n}\sum_{i=1}^n \left\{Y_i - m(X_i)\right\}^2 
\quad \text{ where } \quad \{x\}_+ = \begin{cases} x &\text{if} \  x \ge 0 \\ 0 &\text{ if } x < 0. \end{cases}. 
$$ 

You should be able to implement this with a very small change to your `CVXR` code.

[^1]: It's called nearly-isotonic regression there. Monotone and isotonic are synonyms.